In [1]:
import scSpace
import scanpy as sc
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import numpy as np
import scipy.io
import scipy.linalg
import sklearn.metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import random
import torch
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.benchmark = False

In [3]:
running_time_all = []

In [4]:
# 2,3,4 subclusters (sim5,9,23 error)
ct_num = [3, 4, 5, 3, 4, 5, 3, 4, 5, 3, 4, 5, 3, 4, 5,
          4, (5+1), 6, (7+3), 4, 5, 6, 7, 4, 5, 6, 7, 4, 5, 6, 7, 4, 5, 6, 7,
          5, 6, 8, 5, 6, 8, (5+29), 6, 8, 5, 6, 8, 5, 6, 8]
sim_id = ['sim1', 'sim4', 'sim7', 'sim11', 'sim14', 'sim17', 'sim21', 'sim24', 'sim27', 'sim31', 'sim34', 'sim37', 'sim41', 'sim44', 'sim47',
          'sim2', 'sim5', 'sim8', 'sim9', 'sim12', 'sim15', 'sim18', 'sim19', 'sim22', 'sim25', 'sim28', 'sim29', 'sim32', 'sim35', 'sim38', 'sim39', 'sim42', 'sim45', 'sim48', 'sim49',
          'sim3', 'sim6', 'sim10', 'sim13', 'sim16', 'sim20', 'sim23', 'sim26', 'sim30', 'sim33', 'sim36', 'sim40', 'sim43', 'sim46', 'sim50']


for i in range(50):
    sc_data_dir = '~/workspace/scSpace/data/[0]simulated_data/original_data/' + sim_id[i] + '_sc_data.csv'
    sc_meta_dir = '~/workspace/scSpace/data/[0]simulated_data/original_data/' + sim_id[i] + '_sc_meta.csv'
    st_data_dir = '~/workspace/scSpace/data/[0]simulated_data/original_data/' + sim_id[i] + '_st_data.csv'
    st_meta_dir = '~/workspace/scSpace/data/[0]simulated_data/original_data/' + sim_id[i] + '_st_meta.csv'
    sc_cogaps_dir = '~/workspace/scSpace/data/scCoGAPS/data/' + sim_id[i] + '_sc_cogaps.csv'
    st_cogaps_dir = '~/workspace/scSpace/data/scCoGAPS/data/' + sim_id[i] + '_st_cogaps.csv'
    
    sc_obj, st_obj = scSpace.load_data(sc_data_path=sc_data_dir, sc_meta_path=sc_meta_dir, 
                                       st_data_path=st_data_dir, st_meta_path=st_meta_dir,
                                       spatial_key=['pseudo_x','pseudo_y'])
    
    # pre-porcess
    sc_obj, st_obj = scSpace.preporcess(sc_adata=sc_obj, st_adata=st_obj, 
                                        st_type='spot', n_features=2000, normalize=True)
    
    sc_cogaps = pd.read_csv(sc_cogaps_dir, index_col=0)
    st_cogaps = pd.read_csv(st_cogaps_dir, index_col=0)
    sc_obj.obsm['TCA'] = np.array(sc_cogaps)
    st_obj.obsm['TCA'] = np.array(st_cogaps)
    
    
    setup_seed(123)
    
    starttime = time.time()

    st_dataloader = scSpace.utils.get_data_loader(st_adata=st_obj, batch_size=128)
    mlp = scSpace.utils.init_model(net=scSpace.models.sample_MLPEncoder(input_size=st_obj.obsm['TCA'].shape[1], common_size=2, hidden_size=128, activation='sigmoid'))
    print('Beginning training encoder for source domain...')
    losses = []
    encoder = scSpace.utils.train(encoder=mlp, losses=losses, data_loader=st_dataloader, lr=0.001, epoch_num=1000, log_epoch=1000)
    print('Encoder for source domain training finished.')  
    encoder.eval()
    sc_tca = np.array(sc_obj.obsm['TCA'])
    predict = encoder(scSpace.utils.make_cuda(torch.tensor(sc_tca).to(torch.float32)))
    pseudo_space = predict.cpu().detach().numpy()
    sc_obj.obsm['pseudo_space'] = pseudo_space
    
    sc_obj = scSpace.spatial_cluster(sc_obj, Ks=10, Kg=20, target_num=ct_num[i])
    
    endtime = time.time()
    running_time = endtime - starttime
    running_time_all.append(running_time)
    
    pseudo_space_dir = '~/workspace/scSpace/data/scCoGAPS/result/' + sim_id[i] + '_pseudo_space.csv'
    scspace_clu_dir = '~/workspace/scSpace/data/scCoGAPS/result/' + sim_id[i] + '_cluster_res.csv'
    
    pd.DataFrame(sc_obj.obsm['pseudo_space']).to_csv(pseudo_space_dir)
    sc_obj.obs.to_csv(scspace_clu_dir)

Loading data...
Data have been loaded.
Data have been pre-processed.
Beginning training encoder for source domain...


100%|██████████| 1000/1000 [00:10<00:00, 96.93it/s]


Epoch [1000/1000]: Batch loss=82.43310546875
Encoder for source domain training finished.
Calculating spatial weights using the neighbours of spots...
Start at res =  0.5 step =  0.1
Res =  0.5 number of clusters =  3
Recommended res =  0.5
Loading data...
Data have been loaded.
Data have been pre-processed.
Beginning training encoder for source domain...


100%|██████████| 1000/1000 [00:09<00:00, 102.32it/s]


Epoch [1000/1000]: Batch loss=49.28687286376953
Encoder for source domain training finished.
Calculating spatial weights using the neighbours of spots...
Start at res =  0.5 step =  0.1
Res =  0.5 number of clusters =  3
Res =  0.6 number of clusters =  3
Res changed to 0.6
Res =  0.7 number of clusters =  3
Res changed to 0.7
Res =  0.7999999999999999 number of clusters =  4
Recommended res =  0.7999999999999999
Loading data...
Data have been loaded.
Data have been pre-processed.
Beginning training encoder for source domain...


...

Epoch [1000/1000]: Batch loss=82.10240936279297
Encoder for source domain training finished.
Calculating spatial weights using the neighbours of spots...
Start at res =  0.5 step =  0.1
Res =  0.5 number of clusters =  4
Res =  0.6 number of clusters =  4
Res changed to 0.6
Res =  0.7 number of clusters =  4
Res changed to 0.7
Res =  0.7999999999999999 number of clusters =  5
Recommended res =  0.7999999999999999
Loading data...
Data have been loaded.
Data have been pre-processed.
Beginning training encoder for source domain...


100%|██████████| 1000/1000 [00:09<00:00, 100.62it/s]


Epoch [1000/1000]: Batch loss=61.52866744995117
Encoder for source domain training finished.
Calculating spatial weights using the neighbours of spots...
Start at res =  0.5 step =  0.1
Res =  0.5 number of clusters =  3
Res =  0.6 number of clusters =  4
Res changed to 0.6
Res =  0.7 number of clusters =  4
Res changed to 0.7
Res =  0.7999999999999999 number of clusters =  4
Res changed to 0.7999999999999999
Res =  0.8999999999999999 number of clusters =  6
Recommended res =  0.8999999999999999
Loading data...
Data have been loaded.
Data have been pre-processed.
Beginning training encoder for source domain...


100%|██████████| 1000/1000 [00:09<00:00, 100.23it/s]


Epoch [1000/1000]: Batch loss=51.767459869384766
Encoder for source domain training finished.
Calculating spatial weights using the neighbours of spots...
Start at res =  0.5 step =  0.1
Res =  0.5 number of clusters =  5
Res =  0.6 number of clusters =  5
Res changed to 0.6
Res =  0.7 number of clusters =  6
Res changed to 0.7
Res =  0.7999999999999999 number of clusters =  6
Res changed to 0.7999999999999999
Res =  0.8999999999999999 number of clusters =  7
Res changed to 0.8999999999999999
Res =  0.9999999999999999 number of clusters =  7
Res changed to 0.9999999999999999
Res =  1.0999999999999999 number of clusters =  7
Res changed to 1.0999999999999999
Res =  1.2 number of clusters =  7
Res changed to 1.2
Res =  1.3 number of clusters =  8
Recommended res =  1.3


In [5]:
pd.DataFrame(running_time_all).to_csv('~/workspace/scSpace/data/scCoGAPS/result/cluster_runtime_sim1_50.csv')

In [9]:
running_time_all = []

In [10]:
# 5,6,7,8,9,10 subclusters (sim53 error)
ct_num = [6, 7, (9+17), 6, 7, 9, 6, 7, (9+1), 6, 7, 9, 6, 7, (9+1),
          7, 8, 10, 7, 8, 10, 7, 8, 10, 7, 8, 10, 7, 8, 10,
          8, 9, 11, 8, 9, 11, 8, 9, (11+19), 8, 9, (11+2), 8, 9, 11,
          9, 10, 12, 9, 10, 12, 9, 10, 12, 9, 10, 12, 9, 10, 12,
          10, 11, 13, 10, 11, 13, 10, 11, (13+17), 10, 11, 13, 10, 11, 13,
          11, 12, 14, 11, 12, 14, 11, 12, 14, 11, 12, 14, 11, 12, 14]
sim_id = ['sim51', 'sim52', 'sim53', 'sim54', 'sim55', 'sim56', 'sim57', 'sim58', 'sim59', 'sim60', 'sim61', 'sim62', 'sim63', 'sim64', 'sim65',
          'sim66', 'sim67', 'sim68', 'sim69', 'sim70', 'sim71', 'sim72', 'sim73', 'sim74', 'sim75', 'sim76', 'sim77', 'sim78', 'sim79', 'sim80',
          'sim81', 'sim82', 'sim83', 'sim84', 'sim85', 'sim86', 'sim87', 'sim88', 'sim89', 'sim90', 'sim91', 'sim92', 'sim93', 'sim94', 'sim95',
          'sim96', 'sim97', 'sim98', 'sim99', 'sim100', 'sim101', 'sim102', 'sim103', 'sim104', 'sim105', 'sim106', 'sim107', 'sim108', 'sim109', 'sim110',
          'sim111', 'sim112', 'sim113', 'sim114', 'sim115', 'sim116', 'sim117', 'sim118', 'sim119', 'sim120', 'sim121', 'sim122', 'sim123', 'sim124', 'sim125',
          'sim126', 'sim127', 'sim128', 'sim129', 'sim130', 'sim131', 'sim132', 'sim133', 'sim134', 'sim135', 'sim136', 'sim137', 'sim138', 'sim139', 'sim140']


for i in range(90):
    sc_data_dir = '~/workspace/scSpace/data/add_subcluster/original_data/' + sim_id[i] + '_sc_data.csv'
    sc_meta_dir = '~/workspace/scSpace/data/add_subcluster/original_data/' + sim_id[i] + '_sc_meta.csv'
    st_data_dir = '~/workspace/scSpace/data/add_subcluster/original_data/' + sim_id[i] + '_st_data.csv'
    st_meta_dir = '~/workspace/scSpace/data/add_subcluster/original_data/' + sim_id[i] + '_st_meta.csv'
    sc_cogaps_dir = '~/workspace/scSpace/data/scCoGAPS/data/' + sim_id[i] + '_sc_cogaps.csv'
    st_cogaps_dir = '~/workspace/scSpace/data/scCoGAPS/data/' + sim_id[i] + '_st_cogaps.csv'
    
    sc_obj, st_obj = scSpace.load_data(sc_data_path=sc_data_dir, sc_meta_path=sc_meta_dir, 
                                       st_data_path=st_data_dir, st_meta_path=st_meta_dir,
                                       spatial_key=['pseudo_x','pseudo_y'])
    
    # pre-porcess
    sc_obj, st_obj = scSpace.preporcess(sc_adata=sc_obj, st_adata=st_obj, 
                                        st_type='spot', n_features=2000, normalize=True)
    
    sc_cogaps = pd.read_csv(sc_cogaps_dir, index_col=0)
    st_cogaps = pd.read_csv(st_cogaps_dir, index_col=0)
    sc_obj.obsm['TCA'] = np.array(sc_cogaps)
    st_obj.obsm['TCA'] = np.array(st_cogaps)
    
    
    setup_seed(123)
    
    starttime = time.time()

    st_dataloader = scSpace.utils.get_data_loader(st_adata=st_obj, batch_size=128)
    mlp = scSpace.utils.init_model(net=scSpace.models.sample_MLPEncoder(input_size=st_obj.obsm['TCA'].shape[1], common_size=2, hidden_size=128, activation='sigmoid'))
    print('Beginning training encoder for source domain...')
    losses = []
    encoder = scSpace.utils.train(encoder=mlp, losses=losses, data_loader=st_dataloader, lr=0.001, epoch_num=1000, log_epoch=1000)
    print('Encoder for source domain training finished.')  
    encoder.eval()
    sc_tca = np.array(sc_obj.obsm['TCA'])
    predict = encoder(scSpace.utils.make_cuda(torch.tensor(sc_tca).to(torch.float32)))
    pseudo_space = predict.cpu().detach().numpy()
    sc_obj.obsm['pseudo_space'] = pseudo_space
    
    sc_obj = scSpace.spatial_cluster(sc_obj, Ks=10, Kg=20, target_num=ct_num[i])
    
    endtime = time.time()
    running_time = endtime - starttime
    running_time_all.append(running_time)
    
    pseudo_space_dir = '~/workspace/scSpace/data/scCoGAPS/result/' + sim_id[i] + '_pseudo_space.csv'
    scspace_clu_dir = '~/workspace/scSpace/data/scCoGAPS/result/' + sim_id[i] + '_cluster_res.csv'
    
    pd.DataFrame(sc_obj.obsm['pseudo_space']).to_csv(pseudo_space_dir)
    sc_obj.obs.to_csv(scspace_clu_dir)

Loading data...
Data have been loaded.
Data have been pre-processed.
Beginning training encoder for source domain...


100%|██████████| 1000/1000 [00:10<00:00, 98.29it/s]


Epoch [1000/1000]: Batch loss=196.15594482421875
Encoder for source domain training finished.
Calculating spatial weights using the neighbours of spots...
Start at res =  0.5 step =  0.1
Res =  0.5 number of clusters =  5
Res =  0.6 number of clusters =  7
Step changed to 0.05
Res =  0.55 number of clusters =  7
Step changed to 0.025
Res =  0.525 number of clusters =  6
Recommended res =  0.525
Loading data...
Data have been loaded.
Data have been pre-processed.
Beginning training encoder for source domain...


100%|██████████| 1000/1000 [00:10<00:00, 98.58it/s]


Epoch [1000/1000]: Batch loss=135.99365234375
Encoder for source domain training finished.
Calculating spatial weights using the neighbours of spots...
Start at res =  0.5 step =  0.1
Res =  0.5 number of clusters =  4
Res =  0.6 number of clusters =  4
Res changed to 0.6
Res =  0.7 number of clusters =  5
Res changed to 0.7
Res =  0.7999999999999999 number of clusters =  5
Res changed to 0.7999999999999999
Res =  0.8999999999999999 number of clusters =  6
Res changed to 0.8999999999999999
Res =  0.9999999999999999 number of clusters =  6
Res changed to 0.9999999999999999
Res =  1.0999999999999999 number of clusters =  6
Res changed to 1.0999999999999999
Res =  1.2 number of clusters =  7
Recommended res =  1.2
Loading data...
Data have been loaded.
Data have been pre-processed.
Beginning training encoder for source domain...


...

Epoch [1000/1000]: Batch loss=138.3069305419922
Encoder for source domain training finished.
Calculating spatial weights using the neighbours of spots...
Start at res =  0.5 step =  0.1
Res =  0.5 number of clusters =  7
Res =  0.6 number of clusters =  9
Res changed to 0.6
Res =  0.7 number of clusters =  11
Recommended res =  0.7
Loading data...
Data have been loaded.
Data have been pre-processed.
Beginning training encoder for source domain...


100%|██████████| 1000/1000 [00:10<00:00, 98.61it/s]


Epoch [1000/1000]: Batch loss=155.33047485351562
Encoder for source domain training finished.
Calculating spatial weights using the neighbours of spots...
Start at res =  0.5 step =  0.1
Res =  0.5 number of clusters =  5
Res =  0.6 number of clusters =  8
Res changed to 0.6
Res =  0.7 number of clusters =  10
Res changed to 0.7
Res =  0.7999999999999999 number of clusters =  9
Res changed to 0.7999999999999999
Res =  0.8999999999999999 number of clusters =  11
Res changed to 0.8999999999999999
Res =  0.9999999999999999 number of clusters =  12
Recommended res =  0.9999999999999999
Loading data...
Data have been loaded.
Data have been pre-processed.
Beginning training encoder for source domain...


100%|██████████| 1000/1000 [00:10<00:00, 99.06it/s]


Epoch [1000/1000]: Batch loss=102.48236083984375
Encoder for source domain training finished.
Calculating spatial weights using the neighbours of spots...
Start at res =  0.5 step =  0.1
Res =  0.5 number of clusters =  9
Res =  0.6 number of clusters =  10
Res changed to 0.6
Res =  0.7 number of clusters =  11
Res changed to 0.7
Res =  0.7999999999999999 number of clusters =  12
Res changed to 0.7999999999999999
Res =  0.8999999999999999 number of clusters =  13
Res changed to 0.8999999999999999
Res =  0.9999999999999999 number of clusters =  14
Recommended res =  0.9999999999999999


In [11]:
pd.DataFrame(running_time_all).to_csv('~/workspace/scSpace/data/scCoGAPS/result/cluster_runtime_sim51_140.csv')